In [19]:
import shutil
import os
RESEARCH_WORK_PATH = "/content/drive/MyDrive/Colab Notebooks/BachelorResearch/"


data_dir = "/content/datas/"
if os.path.exists(data_dir):
  shutil.rmtree(data_dir)
for dirs in os.listdir(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/"):
  if "Q" in dirs:
    shutil.copytree(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + dirs, data_dir + dirs)

In [22]:
import librosa
import numpy as np
import torch
import pickle
from tqdm import tqdm


def min_max(x): # do min_max normalization ?->https://atmarkit.itmedia.co.jp/ait/articles/2110/07/news027.html
  min = 0
  max = 154.28702 
  # max = 137.74599
  x = np.abs(x)
  result = (x-min)/(max-min)
  return result

def feature(song_path):
  y, sr = librosa.load(song_path)
  
  n_fft = 512
  hop_length = 256
  stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
  stft = min_max(stft)
  stft = np.expand_dims(stft, axis=2)
  stft = torch.from_numpy(stft)
  stft = stft.permute(2,1,0)

  tempo = librosa.feature.tempogram(y, sr=sr)
  tempo = np.expand_dims(tempo, axis=2)
  tempo = torch.from_numpy(tempo)
  tempo = tempo.permute(2,1,0)
  tempo = tempo.to(torch.float32)

  label = int(song_path.split("/")[-1].split(".")[-3][1])-1
  return [tempo, stft], label

def path_to_audiofiles(dir_folder):
  list_of_audio = []
  for file in os.listdir(dir_folder):
    if file.endswith(".wav"):
      directory = "%s%s" % (dir_folder, file)
      list_of_audio.append(directory)
  return list_of_audio


dst_dir = "/content/dst/"
if not os.path.exists(dst_dir):
  os.mkdir(dst_dir)
  os.mkdir(dst_dir+"Q1")
  os.mkdir(dst_dir+"Q2")
  os.mkdir(dst_dir+"Q3")
  os.mkdir(dst_dir+"Q4")

Qs = ['Q1', 'Q2', 'Q3', 'Q4']

for q in Qs:
  for a in tqdm(path_to_audiofiles(data_dir + q + "/")):
    x,y = feature(a)
    with open(dst_dir + q + '/' + a.split("/")[-1] + '.pickle', mode='wb') as f:
      pickle.dump([x,y], f)


100%|██████████| 2800/2800 [05:18<00:00,  8.79it/s]


In [23]:
for q in "Q1 Q2 Q3 Q4".split(" "):
  d = "/content/datas/" + q + "/"
  print(q, sum(os.path.isfile(os.path.join(d, name)) for name in os.listdir(d)))

Q1 2800
Q2 2800
Q3 2800
Q4 2800


In [24]:
shutil.rmtree(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/pickles2/")

In [25]:
shutil.copytree(dst_dir, RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/pickles2/")

'/content/drive/MyDrive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/pickles2/'

In [26]:
for q in "Q1 Q2 Q3 Q4".split(" "):
  d = RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/pickles2/" + q + "/"
  print(q, sum(os.path.isfile(os.path.join(d, name)) for name in os.listdir(d)))

Q1 2800
Q2 2800
Q3 2800
Q4 2800
